In [1]:
import nnlibrary.configs.hvac_mode_classifier as cfg
from nnlibrary.engines import Trainer

In [2]:
trainer = Trainer(cfg=cfg)


HDF5 Dataset Info for train.h5:
  Input chunks: (100, 48, 16) (chunk = 0.29 MB)
  Output chunks: (100, 3) (chunk = 0.00 MB)

  Loading entire dataset into memory from /home/ahojrup/GitLab/mpc_ahu_neural_network/data/14days_2025-09-09_2025-09-23/dataset/train.h5...
    Expected memory usage:
      Inputs:  (16694, 48, 16) × float32 = 0.048 GB
      Outputs: (16694, 3) × float32 = 0.000 GB
      Total:   0.048 GB

  Actual memory allocated: 0.051 GB
  Loaded 16,694 samples into memory
  Effective compression ratio: 0.95x




wandb: Currently logged in as: andreas-hovaldt (be-ics-acs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [3]:
trainer.train()

  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: Expected `device_type` of type `str`, got: `<class 'torch.device'>`

In [ ]:
len(trainer.trainloader)

33

In [3]:
import torch
from torchvision.transforms import v2
import numpy as np
import sys; sys.path.append('..')
from nnlibrary.utils.operations import Standardize
        


means: np.ndarray = np.load("/home/ahojrup/GitLab/mpc_ahu_neural_network/data/730days_2023-09-24_2025-09-23/dataset-regression/stats/target_mean.npy")
stds: np.ndarray = np.load("/home/ahojrup/GitLab/mpc_ahu_neural_network/data/730days_2023-09-24_2025-09-23/dataset-regression/stats/target_std.npy")
test_target = torch.as_tensor(data=[[5,5,5,5,5],[5,5,5,5,5]], dtype=torch.float32).unsqueeze(dim=0)

transform = v2.Compose([
    Standardize(mean=means.astype(float).tolist(), std=stds.astype(float).tolist()),
])

transformed_target = transform(test_target)
print(transformed_target, "\n\n", transformed_target.shape, "\n\n")

transform = Standardize(mean=means.astype(float).tolist(), std=stds.astype(float).tolist())
transformed_target = transform(test_target)
print(transformed_target, "\n\n", transformed_target.shape, "\n\n")

tensor([[[-0.4045, -1.4380, -6.0174, -3.8171, -5.6814],
         [-0.4045, -1.4380, -6.0174, -3.8171, -5.6814]]]) 

 torch.Size([1, 2, 5]) 


tensor([[[-0.4045, -1.4380, -6.0174, -3.8171, -5.6814],
         [-0.4045, -1.4380, -6.0174, -3.8171, -5.6814]]]) 

 torch.Size([1, 2, 5]) 




In [1]:
import sys; sys.path.append('..')
from nnlibrary.datasets import MpcDatasetHDF5
from pathlib import Path
from nnlibrary.utils.operations import Standardize
import numpy as np

dataset_dir = Path("/home/ahojrup/GitLab/mpc_ahu_neural_network/data/730days_2023-09-24_2025-09-23/dataset-regression-mode")
standardize_transform = Standardize(
    mean=np.load(dataset_dir / "stats" / "target_mean.npy").astype(float).tolist(),
    std=np.load(dataset_dir / "stats" / "target_std.npy").astype(float).tolist(),
)

dataset = MpcDatasetHDF5(
    hdf5_file=dataset_dir / "train.h5",
    task='regression',
    target_transform=None,
)
x, y = dataset.__getitem__(index=0)
print(x.shape, y.shape)

dataset_standardized = MpcDatasetHDF5(
    hdf5_file=dataset_dir / "train.h5",
    task='regression',
    target_transform=standardize_transform,
)
x_std, y_std = dataset_standardized.__getitem__(index=0)
print(x_std.shape, y_std.shape)

torch.Size([32, 14]) torch.Size([5])
Applied target_transform!
torch.Size([32, 14]) torch.Size([5])


In [4]:
print(y, y_std)
print(y.dtype, y_std.dtype)
print(type(y), type(y_std))


print(standardize_transform(y))

tensor([ 1.0000, 77.2230, 15.0000, 14.3489, 14.3366]) tensor([-0.5630,  0.4159, -0.9124, -1.0703, -1.1158])
torch.float32 torch.float32
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([-0.5630,  0.4159, -0.9124, -1.0703, -1.1158])
